In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from LAb_UW_functions import *
from scipy.signal import find_peaks
import glob

In [13]:
def find_mechanical_data(file_path_list, pattern):
    """
    Trova un file specifico all'interno di una lista di percorsi dei file utilizzando un pattern.
    
    Args:
        file_path_list (list): Lista di percorsi dei file in cui cercare il file.
        pattern (str): Pattern per il nome del file da cercare.
    
    Returns:
        str: Percorso completo del file trovato, o None se non viene trovato nessun file corrispondente.
    """
    for file_path in file_path_list:
        if glob.fnmatch.fnmatch(file_path, pattern):
            print("MECHANICAL DATA CHOOSE:", file_path)
            return file_path
    return None  # Nessun file trovato nella lista
    

def find_sync_values(mech_data_path):
    """
    Trova i valori di picco sincronizzazione all'interno di un file di dati meccanici.

    Questa funzione legge un file CSV contenente dati meccanici, estrae la colonna
    relativa alla sincronizzazione e individua i picchi di sincronizzazione in base
    ai parametri specificati.

    Args:
        mech_data_path (str): Percorso del file CSV contenente i dati meccanici.

    Returns:
        numpy.ndarray: Un array NumPy contenente gli indici dei picchi di sincronizzazione
                       trovati nei dati meccanici.
    """
    mech_data = pd.read_csv(mech_data_path, sep=',', skiprows=[1])
    sync_data = mech_data.sync
    
    # Trova i picchi di sincronizzazione nei dati sincronizzazione
    sync_peaks, _ = find_peaks(sync_data, prominence=4.2, height=4)
    print(sync_peaks)
    return mech_data, sync_data, sync_peaks

In [66]:
machine_name = 'Brava_2'
experiment_name = 's0108'
data_type_mech = 'mechanical_data'
sync_file_pattern = '*s*_data_rp' #pattern to find specific experiment in mechanical data

data_type_uw = 'data_tsv_files'
remove_initial_samples = 50

#MAKE MECHANICAL AND UW PATH LIST
infile_path_list_mech = make_infile_path_list(machine_name, experiment_name, data_type=data_type_mech)
infile_path_list_uw = sorted(make_infile_path_list(machine_name,experiment_name, data_type=data_type_uw))

#LOAD MECHANICAL DATA
mech_data_path= find_mechanical_data(infile_path_list_mech, sync_file_pattern)


uw_path = infile_path_list_uw[8]
uw_path


MECHANICAL DATA CHOOSE: c:\Users\miche\Documents\Università\PhD\ACTIVE_SOURCE\experiments_Brava_2\s0108\mechanical_data\s0108sw06car102030_data_rp


'c:\\Users\\miche\\Documents\\Università\\PhD\\ACTIVE_SOURCE\\experiments_Brava_2\\s0108\\data_tsv_files\\009_step_1_100_1_20MPa.bscan.tsv'

In [67]:
mech_data, sync_data, sync_peaks = find_sync_values(mech_data_path)

[  5582   8698  14897  20041  21539  25547  28955  65521  80497 116274
 126306 128395 133717 135661 162041 198593 213558 249224 259432 261425
 266238 270468 273597 309133 324191 359786 369610 374824]


In [68]:
data_uw, metadata_uw = make_UW_data(uw_path)

In [69]:
time_ax_waveform = metadata_uw['time_ax_waveform']
start_time, end_time = time_ax_waveform[0], time_ax_waveform[-1]
sampling_rate = metadata_uw['sampling_rate']
number_of_samples = metadata_uw['number_of_samples']
acquisition_frequency =  metadata_uw['acquition_frequency']
amp_scale = 100

In [148]:
time_plot = mech_data.time_s[sync_peaks[16]:sync_peaks[17]]
time_plot = time_plot - time_plot[sync_peaks[16]]
friction_plot = mech_data.friction_na[sync_peaks[16]:sync_peaks[17]]
displacement_plot = mech_data.lp_disp_mm[sync_peaks[16]:sync_peaks[17]]

time_figure= mech_data.time_s[220793:223192]
#time_figure = time_figure - time_figure[220793]
friction_figure = mech_data.friction_na[220793:223192]
# Calcola il range attuale di friction_plot
min_friction = np.min(friction_plot)
max_friction = np.max(friction_plot)

# Definisce il nuovo intervallo basato su 'Wf_time'
min_wf_time = 29
max_wf_time = 31

# Scala friction_plot per adattarlo all'intervallo di Wf_time
friction_scaled = ((friction_figure - min_friction) / (max_friction - min_friction)) * (max_wf_time - min_wf_time) + min_wf_time

In [152]:
%matplotlib qt

fig,ax1 = plt.subplots(ncols = 1,figsize=(15,8))
ax1 = plt.subplot()
ax1.set_title("Waveforms Amplitude map VS Friction data" , fontsize = 16)
cmap = plt.get_cmap('seismic')

#ax1.plot(time_plot, friction_scaled)
ax1.plot(time_figure, friction_scaled, linewidth = 1.5, c = 'k')
im = ax1.imshow(data_uw[7235:9634].T, aspect='auto',origin='lower',interpolation='none',
                cmap = cmap, vmin = -amp_scale, vmax = amp_scale,
                extent = [0,23.98, 0 , number_of_samples*sampling_rate])

cbar= fig.colorbar(im,pad=0.04)
cbar.set_label("Relative Amplitude [a.u.]", fontsize = 14)
ax1.set_xlabel('Time [s]', fontsize = 14)
ax1.set_ylabel('Wf_time [$\mu s$] ', fontsize = 14)
ax1.set_xscale
ax1.set_ylim(28,32)
ax1.tick_params(axis='x', labelsize=12)  
ax1.tick_params(axis='y', labelsize=12)
cbar.ax.tick_params(labelsize=12)
#fig.tight_layout()

#plt.savefig('amplitudemap_on_vstep_1.pdf', dpi = 300)


In [123]:
plt.plot(mech_data.time_s[220793:223192],mech_data.friction_na[220793:223192])

In [150]:
time_figure

220793    11483.655
220794    11483.665
220795    11483.675
220796    11483.685
220797    11483.695
            ...    
223187    11507.595
223188    11507.605
223189    11507.615
223190    11507.625
223191    11507.635
Name: time_s, Length: 2399, dtype: float64